<a href="https://colab.research.google.com/github/shashithenuwara/IRWA_Project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2 pdfplumber spacy nltk whoosh transformers torch beautifulsoup4
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from PyPDF2 import PdfReader

#import files to the colab environment
pdf_directory = '/content/drive/My Drive/Colab Notebooks/MIniDataSet'

# extract texts from all PDFs
text_data = ""
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + "\n"




In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

# Tokenization
tokens = word_tokenize(text_data)

# Normalization (Lowercasing, Removing punctuation, Stopwords)
tokens = [word.lower() for word in tokens if word.isalpha()]
tokens = [word for word in tokens if word not in stopwords.words('english')]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import spacy

nlp = spacy.load('en_core_web_sm')

# Increase the max_length limit
nlp.max_length = len(text_data)

doc = nlp(text_data)

for ent in doc.ents:
    print(ent.text, ent.label_)

Streaming output truncated to the last 5000 lines.
miR-124-3p ORG
STAT3 ORG
EMT ORG
Ji et al. PERSON
2019 DATE
Lv et al. PERSON
2011 DATE
124 CARDINAL
Wang ORG
al. GPE
2016a DATE
124 CARDINAL
CDYL2 PRODUCT
STAT3 ORG
CDYL2 PRODUCT
CDYL2 PRODUCT
CDYL2 LAW
STAT3 ORG
CDYL2 PRODUCT
STAT3 ORG
miR- ORG
CDYL2 PRODUCT
STAT3 ORG
miR-124-3p ORG
STAT3 ORG
CDYL2 ORG
miR-124-3p ORG
CDYL2 PRODUCT
kB PERSON
BIRC3 WORK_OF_ART
STAT3 ORG
CDYL2 PRODUCT
g u l ORG
Cao NORP
al. GPE
2018 DATE
Mehta PERSON
al. GPE
2017 DATE
Wang ORG
al. GPE
STAT3 ORG
CDYL2 PRODUCT
kB PERSON
STAT3 ORG
Banerjee ORG
Resat ORG
2016 DATE
Huber et al. PERSON
2004 DATE
p . GPE
Wu PERSON
al. GPE
2009 DATE
CDYL2 PRODUCT
CDYL2 PRODUCT
CDYL2 ORG
MDA ORG
CDYL2 PRODUCT
MDA ORG
7 CARDINAL
CDYL2 PRODUCT
G9a GPE
MIR124-2 FAC
MCF7-CDYL2 ORG
MCF7-Vector ORG
IgG ORG
ChIP ORG
three CARDINAL
H3K9me2, H3K27me3 ORG
hromatin PERSON
MDA ORG
F) Schematic PRODUCT
CDYL2 PRODUCT
MIR124 GPE
ACCESS
iScience PRODUCT
23, 101141 DATE
June 26, 2020 15iScienc DA

In [6]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Stemming
stemmer = PorterStemmer()
stems = [stemmer.stem(word) for word in tokens]

# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(word) for word in tokens]


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
import os

# Define schema
schema = Schema(title=TEXT(stored=True), content=TEXT)

# Create index directory
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# Create an index
ix = create_in("indexdir", schema)

# Indexing documents (Assuming titles are available for each document)
writer = ix.writer()

# Add documents to index
writer.add_document(title="Document 1", content="This is the first document.")
writer.add_document(title="Document 2", content="This is the second document.")
writer.commit()


In [8]:
# Indexing documents
writer = ix.writer()
# Add documents to index (replace with your actual PDF titles and content)
for i, filename in enumerate(os.listdir(pdf_directory)):
    if filename.endswith(".pdf"):
        with open(os.path.join(pdf_directory, filename), 'rb') as file:
            reader = PdfReader(file)
            content = ""
            for page in reader.pages:
                content += page.extract_text() + "\n"
            writer.add_document(title=filename, content=content)
writer.commit()

In [9]:
from whoosh.qparser import QueryParser
from whoosh.index import open_dir # import the open_dir function

ix = open_dir("indexdir") # Call the correct function
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("document")
    results = searcher.search(query)
    for result in results:
        print(result['title'])

Document 1
Document 2


In [10]:
from nltk.corpus import wordnet

# Get synonyms for query expansion
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return set(synonyms)

query = "happy"
expanded_query = query + " " + " ".join(get_synonyms(query))
print(expanded_query)


happy felicitous well-chosen glad happy


In [11]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.4 MB/s eta 0:00:00


In [12]:
from sentence_transformers import SentenceTransformer, util

# Load BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Semantic search
query = "What is AI?"
query_embedding = model.encode(query)

documents = ["Document about artificial intelligence.", "Text related to machine learning."]
doc_embeddings = model.encode(documents)

# Find best match
hits = util.semantic_search(query_embedding, doc_embeddings)
print(hits)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[{'corpus_id': 0, 'score': 0.5344616174697876}, {'corpus_id': 1, 'score': 0.22655534744262695}]]


In [13]:
from whoosh import query

ix = open_dir("indexdir")
with ix.searcher() as searcher:
    fuzzy_query = query.FuzzyTerm("content", "document", maxdist=1)
    results = searcher.search(fuzzy_query)
    for result in results:
        print(result['title'])


Document 1
Document 2


In [14]:
!pip install streamlit
import streamlit as st
from whoosh.qparser import QueryParser
from whoosh.index import open_dir

# Streamlit app for search engine
st.title("Search Engine")

query = st.text_input("Enter search query:")

if query:
    ix = open_dir("indexdir")
    with ix.searcher() as searcher:
        parser = QueryParser("content", ix.schema)
        parsed_query = parser.parse(query)
        results = searcher.search(parsed_query)

        for result in results:
            st.write(result['title'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


2024-10-05 05:13:08.514 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.426 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-05 05:13:09.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 05:13:09.469 Session state does not 

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predicted and actual results
y_true = [1, 0, 1, 1, 0]
y_pred = [1, 0, 1, 0, 0]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Precision: {precision}, Recall: {recall}, F1-Score: {f1}')


Precision: 1.0, Recall: 0.6666666666666666, F1-Score: 0.8


In [16]:
!pip install streamlit pyngrok joblib


In [23]:
%%writefile app.py
import streamlit as st
from whoosh.index import open_dir
from whoosh.qparser import QueryParser
from PyPDF2 import PdfReader
import os

pdf_directory = '/content/drive/My Drive/Colab Notebooks/MIniDataSet'

def main():
    global pdf_directory
    st.title("PDF Search Engine")
    query = st.text_input("Enter your search query:")

    if st.button("Search"):
        if query:
            ix = open_dir("indexdir")
            with ix.searcher() as searcher:
                parser = QueryParser("content", ix.schema)
                parsed_query = parser.parse(query)
                results = searcher.search(parsed_query)

                st.write("Search Results:")
                if results:
                    for result in results:
                        with st.expander(result['title']):
                            pdf_path = os.path.join(pdf_directory, result['title'])
                            with open(pdf_path, 'rb') as file:
                                reader = PdfReader(file)
                                for page in reader.pages:
                                    st.write(page.extract_text())
                else:
                    st.write("No results found.")
        else:
            st.write("Please enter a query to search.")

if __name__ == '__main__':
    main()

Overwriting app.py


In [25]:
from pyngrok import ngrok
import os

# Kill any existing Streamlit processes
os.system('pkill streamlit')

# Run the Streamlit app
get_ipython().system_raw('streamlit run app.py &')

# Create a public URL using ngrok
!ngrok authtoken 2mhmVgSRpnqiFyZL3WoY4j0EA7H_6MuP3KYnr2QCVN3f2tYZ
url = ngrok.connect(addr='http://localhost:8501')
print(f"Streamlit app is live at: {url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://5896-34-138-101-74.ngrok-free.app" -> "http://localhost:8501"
